In [107]:
import networkx as nx
import csv

grafo = nx.Graph()

In [108]:
from SPARQLWrapper import SPARQLWrapper, JSON, RDF
from rdflib import Graph 
import json

#endPoint = "http://172.17.16.218:7200/repositories/TFT"
#endPoint = "http://DESKTOP-TTM16LQ:7200/repositories/GrafoResultanteFinal"
endPoint = "http://DESKTOP-TTM16LQ:7200/repositories/TFT"
sparql = SPARQLWrapper(endPoint)

def getResults(keyword):
    SimilarityKeywords = """
    PREFIX :<http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst:<http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX pubo: <http://ontology.ontotext.com/publishing#>
    PREFIX schema: <http://schema.org/>

    SELECT ?documentID ?documentSec ?score {
    ?search a inst:SSTAK ;
        :searchTerm "%s";
        :searchParameters "";
        :documentResult ?result .
    ?result :value ?documentID ;
            :score ?score.
    ?documentID schema:identifier ?documentSec
    } LIMIT 20
    """
    resultList = []
    sparql.setQuery(SimilarityKeywords%(keyword))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        resultList.append([result['documentID']['value'], result['documentSec']['value'], result['score']['value']])
    return resultList
        

In [23]:
#resultados despues de ejecutar la funcion
results = getResults("Semantic Web")
results

[['https://w3id.org/scholarlydata/inproceedings/eswc2007/paper-179',
  '223',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproceedings/eswc2007/paper-236',
  '233',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproceedings/eswc2007/paper-239',
  '235',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproceedings/eswc2007/paper-298',
  '249',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproceedings/eswc2007/paper-4',
  '255',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproceedings/eswc2007/paper-58',
  '258',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproceedings/eswc2007/paper-7',
  '261',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproceedings/eswc2007/paper-87',
  '264',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproceedings/eswc2007/paper-96',
  '267',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproceedings/eswc2008/paper/11',
  '273',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproceedings/eswc2008/paper/120',
  '277',
  '1.0'],
 ['https://w3id.org/scholarlydata/inproc

In [109]:
def searchPapersIndexPredic(idPaper):
    SSpapers = """
    PREFIX : <http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst: <http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX schema: <http://schema.org/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX skg: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    PREFIX pred: <http://www.ontotext.com/graphdb/similarity/psi/>
    
    SELECT distinct ?entity ?id ?title ?description (GROUP_CONCAT(DISTINCT ?keywords; SEPARATOR=",") AS ?keys)
            (GROUP_CONCAT(DISTINCT ?author; SEPARATOR=",") AS ?authors) ?score WHERE {
        ?search a inst:PredicationIndex ; psi:searchEntity <%s>; psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
            :searchParameters ""; psi:entityResult ?result .
        ?result :value ?entity ; :score ?score .
        ?entity rdf:type schema:Article ; schema:identifier ?id .
        OPTIONAL { ?entity <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#title> ?title .}
        OPTIONAL { ?entity <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#abstract> ?description . }
        OPTIONAL { ?entity schema:mentions/rdfs:label|dct:subject/rdfs:label|skg:keyword ?keywords . }
        OPTIONAL { ?entity dct:creator ?author . }
    }group by ?entity ?id ?title ?description ?score
    """
    resultList = []
    sparql.setQuery(SSpapers%(idPaper))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        resultList.append([result['entity']['value'], result['id']['value'], result['title']['value'], result['score']['value'], result['keys']['value']])
        #no permite sacar la variable "description"
    return resultList

In [25]:
#resultados despues de ejecutar la funcion
#results = searchPapers("https://w3id.org/scholarlydata/inproceedings/eswc2008/paper/186")
results = searchPapersIndexPredic("https://w3id.org/scholarlydata/inproceedings/iswc-2019-challenge-25")


results

[['https://w3id.org/scholarlydata/inproceedings/iswc-2019-challenge-25',
  '4842',
  'Fact Validation Challenge',
  '0.9999999928150064',
  'Semantic Web,Semantic  Web,SemanticW,RDF,Triples,DrugBank,GERBIL,ISWC,HOBBIT,Knowledge representation,Metadata,Technical communication,XML-based standards,Buzzwords,Web services,Resource Description Framework,World Wide Web Consortium standards,XML,Bibliography file formats,Baseball terminology,Batting statistics,ISO standards,Information science,Ontology (information science),Identifiers,Knowledge engineering,Chemical databases,Metabolomic databases,Muridae,Gerbils,Pet rodents,Fauna of Central Asia,Fauna of Mongolia,Universal identifiers,Middle-earth races,Fictional species and races,Middle-earth Hobbits,Fact validation,Large-scale benchmarks,Machine learning,Knowledge extraction'],
 ['https://w3id.org/scholarlydata/inproceedings/iswc-2018-poster_and_demo-364',
  '5488',
  'Querying RDF data in Networks of Web Browsers',
  '0.3950618007519712',
 

In [26]:
#2.1 Indice basado en metadatos del dataset original (titulo. keywords, description, autores)
def indice1(idPaper):
    SSpapers = """
    PREFIX : <http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst: <http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX schema: <http://schema.org/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX skg: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    PREFIX pred: <http://www.ontotext.com/graphdb/similarity/psi/>
    
    SELECT distinct ?entity ?title (GROUP_CONCAT(DISTINCT ?keywords; SEPARATOR=",") AS ?keys) ?description 
            (GROUP_CONCAT(DISTINCT ?author; SEPARATOR=",") AS ?authors) ?score WHERE {
        ?search a inst:PredicationIndex ; psi:searchEntity <%s>; 
        psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
            psi:entityResult ?result .
        ?result :value ?entity ; 
            :score ?score.
        ?entity rdf:type schema:Article .
        OPTIONAL { ?entity skg:title ?title .}
        OPTIONAL { ?entity skg:abstract ?description . }
        OPTIONAL { ?entity skg:keyword ?keywords . }
        OPTIONAL { ?entity dct:creator ?author . }
    }group by ?entity ?title ?description ?score
    """
    resultList = []
    sparql.setQuery(SSpapers%(idPaper))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        resultList.append([result['entity']['value'], result['title']['value'], result['keys']['value'], result['authors']['value'], result['score']['value']])
        #no permite sacar la variable "description"
    return resultList

In [27]:
#resultados despues de ejecutar la funcion
#results = indice1("https://w3id.org/scholarlydata/inproceedings/eswc2008/paper/186")
results = indice1("https://w3id.org/scholarlydata/inproceedings/iswc-2019-challenge-25")

results

[['https://w3id.org/scholarlydata/inproceedings/iswc-2019-challenge-25',
  'Fact Validation Challenge',
  'Fact validation,Large-scale benchmarks,Machine learning,Knowledge extraction',
  'https://w3id.org/scholarlydata/person/zafar-habeed-syed,https://w3id.org/scholarlydata/person/axel-cyrille-ngonga-ngomo,https://w3id.org/scholarlydata/person/michael-roder',
  '0.9999999928150064'],
 ['https://w3id.org/scholarlydata/inproceedings/iswc-2018-poster_and_demo-364',
  'Querying RDF data in Networks of Web Browsers',
  '',
  '',
  '0.3950618007519712'],
 ['https://w3id.org/scholarlydata/inproceedings/iswc2013/data.semanticweb.org/workshop/wasabi/2013/proceedings/paper-04',
  'Boosting RDF Adoption in Ruby with Goo',
  '',
  '',
  '0.36436205634298635'],
 ['https://w3id.org/scholarlydata/inproceedings/iswc2005/proceedings/paper-19',
  'Combining RDF and Part of OWL with Rules: Semantics, Decidability, Complexity',
  '',
  '',
  '0.35937165322259157'],
 ['https://w3id.org/scholarlydata/inpro

In [28]:
#2.2 Indice basado en metadatos base + dbr, dbc (schema:mentions, dct:subject)
def indice2(idPaper):
    SSpapers = """
    PREFIX : <http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst: <http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX schema: <http://schema.org/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX skg: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    PREFIX pred: <http://www.ontotext.com/graphdb/similarity/psi/>
    
    SELECT distinct ?entity ?title ?description 
    (GROUP_CONCAT(DISTINCT ?dbr1; SEPARATOR=",") AS ?dbr) 
    (GROUP_CONCAT(DISTINCT ?dbc1; SEPARATOR=",") AS ?dbc) 
    (GROUP_CONCAT(DISTINCT ?keywords; SEPARATOR=",") AS ?keys) 
    (GROUP_CONCAT(DISTINCT ?author; SEPARATOR=",") AS ?authors) ?score
WHERE {
        ?search a inst:PredicationIndex; psi:searchEntity <%s>; psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
            :searchParameters ""; psi:entityResult ?result .
        ?result :value ?entity ;
                :score ?score.
        ?entity rdf:type schema:Article .
        OPTIONAL { ?entity skg:title ?title .}
        OPTIONAL { ?entity skg:abstract ?description . }
        OPTIONAL { ?entity skg:keyword ?keywords . }
        OPTIONAL { ?entity dct:creator ?author . }
        OPTIONAL { ?entity schema:mentions/rdfs:label ?dbr1 . }
        OPTIONAL { ?entity dct:subject/rdfs:label ?dbc1 . }
    }group by ?entity ?title ?description ?score
    """
    resultList = []
    sparql.setQuery(SSpapers%(idPaper))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        resultList.append([result['entity']['value'], result['title']['value'], result['keys']['value'], result['dbr']['value'], result['dbc']['value'], result['score']['value']])
        #no permite sacar la variable "description"
    return resultList

In [29]:
#resultados despues de ejecutar la funcion
#results = indice2("https://w3id.org/scholarlydata/inproceedings/eswc2008/paper/186")
results = indice2("https://w3id.org/scholarlydata/inproceedings/iswc-2019-challenge-25")

results

[['https://w3id.org/scholarlydata/inproceedings/iswc-2019-challenge-25',
  'Fact Validation Challenge',
  'Fact validation,Large-scale benchmarks,Machine learning,Knowledge extraction',
  'Semantic Web,Semantic  Web,SemanticW,RDF,Triples,DrugBank,GERBIL,ISWC,HOBBIT',
  'Knowledge representation,Metadata,Technical communication,XML-based standards,Semantic Web,Buzzwords,Web services,Resource Description Framework,World Wide Web Consortium standards,XML,Bibliography file formats,Baseball terminology,Batting statistics,ISO standards,Information science,Ontology (information science),Identifiers,Knowledge engineering,Chemical databases,Metabolomic databases,Muridae,Gerbils,Pet rodents,Fauna of Central Asia,Fauna of Mongolia,Universal identifiers,Middle-earth races,Fictional species and races,Middle-earth Hobbits',
  '0.9999999928150064'],
 ['https://w3id.org/scholarlydata/inproceedings/iswc-2018-poster_and_demo-364',
  'Querying RDF data in Networks of Web Browsers',
  '',
  'RDF',
  'Know

In [ ]:
#2.3 Indice completo metadatos base + dbr + dbc + spacy
def indice3(idPaper):
    SSpapers = """
    PREFIX : <http://www.ontotext.com/graphdb/similarity/>
    PREFIX inst: <http://www.ontotext.com/graphdb/similarity/instance/>
    PREFIX psi: <http://www.ontotext.com/graphdb/similarity/psi/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX schema: <http://schema.org/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX dct: <http://purl.org/dc/terms/>
    PREFIX skg: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    PREFIX pred: <http://www.ontotext.com/graphdb/similarity/psi/>
    
    SELECT distinct ?entity ?title ?description 
(GROUP_CONCAT(DISTINCT ?spacySu; SEPARATOR=",") AS ?spacySujeto)
(GROUP_CONCAT(DISTINCT ?spacyPre; SEPARATOR=",") AS ?spacyPredicado)
(GROUP_CONCAT(DISTINCT ?spacyOb; SEPARATOR=",") AS ?spacyObjeto)
(GROUP_CONCAT(DISTINCT ?dbr1; SEPARATOR=",") AS ?dbr) 
(GROUP_CONCAT(DISTINCT ?dbc1; SEPARATOR=",") AS ?dbc) 
(GROUP_CONCAT(DISTINCT ?keywords; SEPARATOR=",") AS ?keys) 
(GROUP_CONCAT(DISTINCT ?author; SEPARATOR=",") AS ?authors) 
WHERE {
        ?search a inst:PredicationIndex; psi:searchEntity <https://w3id.org/scholarlydata/inproceedings/dc-2010/paper/main/A-1005>; psi:searchPredicate <http://www.ontotext.com/graphdb/similarity/psi/any>;
            :searchParameters ""; psi:entityResult ?result .
        ?result :value ?entity . 
        ?entity rdf:type schema:Article .
        ?entity schema:hasPart/schema:about ?spacyS .
        ?spacyS schema:name ?spacySu .
        ?entity schema:hasPart/schema:about/schema:subjectOf ?spacyP .
        ?spacyP schema:name ?spacyPre .
        OPTIONAL { ?entity skg:title ?title .}
        OPTIONAL { ?entity skg:abstract ?description . }
        OPTIONAL { ?entity skg:keyword ?keywords . }
        OPTIONAL { ?entity dct:creator ?author . }
        OPTIONAL { ?entity schema:mentions/rdfs:label ?dbr1 . }
        OPTIONAL { ?entity dct:subject/rdfs:label ?dbc1 . }
    
    }group by ?entity ?title ?description
    """
    resultList = []
    sparql.setQuery(SSpapers%(idPaper))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        resultList.append([result['entity']['value'], result['id']['value'], result['title']['value'], result['score']['value'], result['keys']['value']])
        #no permite sacar la variable "description"
    return resultList

# Load data from ids file:

In [110]:
import networkx as nx
import csv
import pandas as pd
import matplotlib.pyplot as plt

#tsvfile = pd.read_csv(open("papersT.tsv", "r"), delimiter="\t", header=None)
tsvfile = pd.read_csv(open("papersD.tsv", "r"), delimiter="\t", header=None)

tsvfile = tsvfile.values.tolist()
tsvfile[0:5]

[[4842, 1, 5935],
 [4842, 1, 6738],
 [4842, 1, 9063],
 [4843, 1, 5737],
 [4843, 1, 7430]]

In [111]:
def verify(e):
    # To select triplets where ids are retrieved from GraphDB
    if e in ids:
        return True
    else:
        return False
    
def createSubGraph():
    # To create the subgraph based on ids of papers:
    subset = [[line[0], line[2]] for line in tsvfile if verify(line[0]) or verify(line[2])]; subset[0:20]
    for line in subset:
        x, y = line
        g.add_edge(x, y)
    #nx.draw_circular(g, with_labels = True)
    #plt.savefig("filename1.png")
    #plt.clf()

def calculateSimm(min=0.2):
    # To compute the simmilarity matrix between selected entities.
    simMatrix = nx.simrank_similarity(g)
    simList = []
    for d in simMatrix:
        for dd in simMatrix[d]:
            if simMatrix[d][dd] >= min and simMatrix[d][dd] < 1:
                simList.append([d, dd, simMatrix[d][dd]])
    return simList

In [112]:
# to run:
#results = getResults("Semantic Web")
results = searchPapersIndexPredic("https://w3id.org/scholarlydata/inproceedings/iswc-2019-challenge-25")

ids = [int(i[1]) for i in results]
print(ids)
g = nx.Graph()
createSubGraph()
print(g)
sim = calculateSimm()
print(sim)

[4842, 5488, 2522, 1620, 5243, 1533, 848, 1566, 1407, 5402, 2583, 1384, 5353, 842, 1791, 2542, 620, 620, 1956, 2360, 2299]



KeyboardInterrupt: 

In [ ]:
# filtra solo papers

print(len(sim)) # 8622

simSel = [[line[0], line[1], line[2]] for line in sim if verify(line[0]) or verify(line[1])]; len(simSel)

In [ ]:
g.nodes()
g.edges()

In [9]:
def getMetadata(idSec):
    searchMetada = """
    PREFIX schema: <http://schema.org/>
    PREFIX skg: <https://w3id.org/scholarlydata/ontology/conference-ontology.owl#>
    select distinct ?entity ?title ?abs 
    (GROUP_CONCAT(DISTINCT ?keywords; SEPARATOR=",") AS ?keys)
    where{  
        ?entity <http://schema.org/identifier> "%s".
        ?entity skg:title ?title.
        OPTIONAL {?entity skg:abstract ?abs . }
        OPTIONAL {?entity skg:keyword ?keywords . }   
    }GROUP BY ?entity ?title ?abs
    """
    resultList = []
    sparql.setQuery(searchMetada%(idSec))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        #resultList.append([result['entity']['value'], result['title']['value'], result['keys']['value'], result['abstract']['value']])
        #abs = (result["abs"]["value"])
        #print(abs,"abs")
        resultList.append([result['entity']['value'], result['title']['value'], result['keys']['value']])

    return resultList
    

In [ ]:
results = getMetadata(2583)
results

In [10]:
#print(simSel)
metadataList1 = []
unoSolo = set()
unoSolo1 = set()
metadataList2 = []
print(simSel, "print1")
print(simSel[0],simSel[1],"print2")
for i in simSel:
    id1 = i[0]
    score = i[2]
    metadataList1.append(id1)
#guardamos un unico dato    
unoSolo = set(metadataList1)
metadataList1 = unoSolo
    
for i in metadataList1:
    metadata1 = getMetadata(i)
    #print(metadata1, "datos columna 1")
    
for x in simSel:
    id2 = x[1]
    score = x[2] 
    metadataList2.append(id2)
unoSolo1 = set(metadataList2)
metadataList2 = unoSolo1

for x in metadataList2:
    metadata2 = getMetadata(x)
    #print (metadata2, "datos de los papers de la columna 2")
    
print(metadataList1,"lista1")
print(metadataList2, "lista2")
    

[[4842, 620, 0.21192610091117586], [4842, 842, 0.2713442893508928], [4842, 848, 0.28748535321114416], [4842, 1791, 0.22341624070038474], [4842, 1956, 0.2008807567770148], [4842, 5488, 0.23360526941188808], [4842, 1384, 0.23594080872685383], [4842, 1407, 0.24213105971604568], [4842, 1533, 0.23360526941188808], [4842, 1566, 0.261177232891715], [4842, 1620, 0.2202984894361493], [4842, 2299, 0.2514908695992273], [4842, 2360, 0.2045564216129244], [4842, 2522, 0.23360526941188808], [4842, 2542, 0.23360526941188808], [4842, 2583, 0.22666916390489641], [4842, 5243, 0.2173047313013234], [4842, 5353, 0.21665398000704078], [4842, 5402, 0.23360526941188808], [620, 4842, 0.21192610091117633], [620, 842, 0.22834857699809546], [620, 1791, 0.2395361483975774], [620, 1956, 0.2852319128850748], [620, 5488, 0.27741222087953193], [620, 1384, 0.2538373331331318], [620, 1407, 0.24637586388709437], [620, 1533, 0.27741222087953193], [620, 1566, 0.2544749078323423], [620, 1620, 0.261691336534763], [620, 2299, 

{2583, 5402, 1566, 1956, 2299, 1791, 2360, 842, 848, 1620, 2522, 1384, 5353, 4842, 620, 2542, 5488, 5243, 1533, 1407} lista1
{2583, 5402, 1566, 1956, 2299, 1791, 2360, 842, 848, 1620, 2522, 1384, 5353, 4842, 620, 2542, 5488, 5243, 1533, 1407} lista2


In [ ]:
import pandas as pd

def getUris(paperId):
    legendUris = []
    unoSolo = set()
    searchSu = """
    select * where { 
        ?s <http://schema.org/identifier> "%s" .
    }
    """
    sparql.setQuery(searchSu%(paperId))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    Tabla = """\
+---------------------------------------+
| Identificador     Uri                 |
|---------------------------------------|
{}
+---------------------------------------+\
"""
  
    for result in results["results"]["bindings"]:
        uri = result['s']['value']
        #print(uri)
        legendUris.append([paperId, uri])
        #legendUris = pd.DataFrame(legendUris, columns = ['Identificador', 'Uris'])
        Tabla = (Tabla.format('\n'.join("| {:<8} {:<10} |".format(*fila) for fila in legendUris)))
        #print(Tabla)
    unoSolo = set(legendUris)
    legendUris = unoSolo
        #legendUris = pd.DataFrame(legendUris, columns = ['Identificador', 'Uris'])

    return legendUris
#https://pythones.net/format-en-python-3/

In [ ]:
#https://www.geeksforgeeks.org/python-visualize-graphs-generated-in-networkx-using-matplotlib/
#https://stackoverflow.com/questions/25639169/networkx-change-color-width-according-to-edge-attributes-inconsistent-result
   
import networkx as nx
grafo = nx.Graph()

for lines in simSel:
    x, y = lines[:2]
    score = lines[2]
    getUris(x)
    getUris(y)
    if score > 0.3:
        score =3
    grafo.add_edge(x, y, color = 'r', weight = score)

#tipo de dibujo
pos = nx.circular_layout(grafo)

#colores y ancho
colors = nx.get_edge_attributes(grafo,'color').values()
weights = nx.get_edge_attributes(grafo,'weight').values()

nx.draw(grafo, pos, edge_color = colors, width = list(weights), with_labels=True)
plt.savefig("filename1.png")

#Pendiente: consulta a grapdb para obtener los metadatos de estos papers

In [ ]:
https://towardsdatascience.com/python-interactive-network-visualization-using-networkx-plotly-and-dash-e44749161ed7

https://pyvis.readthedocs.io/en/latest/

In [ ]:
from pyvis.network import Network

g = Network()
g.add_node(0)
g.add_node(1)
g.add_edge(0, 1)
g.show("basic.html")

# Visualizacion con Network

In [123]:
#https://pyvis.readthedocs.io/en/latest/
def getUris(paperId):
    legendUri = []
    searchSu = """
    select * where { 
        ?s <http://schema.org/identifier> "%s" .
    }
    """
    sparql.setQuery(searchSu%(paperId))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    for result in results["results"]["bindings"]:
        uri = result['s']['value']
        legendUri.append([paperId, uri])
    return legendUri
    

from pyvis.network import Network
#import networkx as nx
#nt = Network()
#net = Network(height='750px', width='100%', bgcolor='#222222', font_color='white')
#net = Network(height='750px', width='100%', font_color='white')
net = Network('900px','900px')
listUri = []
unoSolo = set()
for lines in simSel:
    x,y = lines[:2]
    score = lines[2]
    #print(x,y,score)
    #print(x)
    if score > 0.3:
        score = 6
    #net.add_node(x, label= "node",borderWidth = score,borderWidthSelected = score, title='nodess', shape = 'https://pyvis.readthedocs.io/en/latest/')
    net.add_node(x)
    net.add_node(y) 
    net.add_edge(x, y, weight=score, width=score,title = score)
    listUri.append(x)
unoSolo = set(listUri)
listUri = unoSolo
#print(listUri,"uno solo")
for i in listUri:
    #print(i,"xxx")
    URIS = getUris(i)
    #print(URIS)
    
#network.Network.set_options()
#net.from_nx(nx.florentine_families_graph())
net.show_buttons(filter_=['physics'])
#nt.set_op
#nt.


In [124]:

net.show("dot.html")
